In [21]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.schema import Document
from langchain.vectorstores import FAISS
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from uuid import uuid4
from langchain_community.docstore.in_memory import InMemoryDocstore
import faiss
import os
load_dotenv()

True

In [3]:
pdf_loader = PyPDFLoader("../dataset/RH_Budi_Casual.pdf")
pages = []
async for page in pdf_loader.alazy_load():
    pages.append(page)

In [4]:
print(f"Terdapat {len(pages)} Halaman")
pages

Terdapat 1 Halaman


[Document(metadata={'source': '../dataset/RH_Budi_Casual.pdf', 'page': 0}, page_content='Nama saya adalah Budi Susanto, saya seorang programmer yang baru lulus dari \nUniversitas Hang Tuah di Surabaya di tahun 2024. Saya mengambil jurusan Teknik \ninformatika Bersama 78 teman saya lainnya. Selama kuliah, saya mempelajari banyak hal \nmengenai pemrograman, algorithma, kompleksitas, eﬁsiensi, kolaboratif, dan lain-lainnya \nyang sangat menantang bagi saya. Hal ini karena saya baru mempunyai personal computer \nsaya 5 tahun yang lalu, dan waktu itu computer adalah hal yang benar-benar baru bagi \nsaya. \n \nSaya lahir di Surabaya, dalam keluarga yang kondisi ekonominya kurang baik, bapak saya \nadalah seorang pedagang asongan yang bekerja keras membanting tulang untuk \nmemenuhi kebutuhan hidup saya sebagai anak tunggalnya. Ibu saya juga pekerja paruh \nwaktu di salah satu usaha laundry milik tetangga. Di rumah, kami juga ditemani oleh \nnenek. Saya sangat dekat dengan nenek saya, karena 

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Ukuran potongan dalam karakter
    chunk_overlap=200  # Jumlah karakter yang tumpang tindih antar potongan
)

In [6]:
documents = []
for page in pages:
    split_doc = text_splitter.split_text(page.page_content)
    uuid = str(uuid4())
    for content in split_doc:
        documents.append(Document(metadata={"parent_id": uuid, "source": page.metadata["source"], "page": page.metadata["page"]}, page_content=content))

In [7]:
uuids = [str(uuid4()) for _ in documents]

In [8]:
print(f"Banyak Chunk dari {len(pages)} Halaman: {len(documents)}")
documents

Banyak Chunk dari 1 Halaman: 3


[Document(metadata={'parent_id': 'c439b32a-9e4e-4805-acef-21a7daa0510e', 'source': '../dataset/RH_Budi_Casual.pdf', 'page': 0}, page_content='Nama saya adalah Budi Susanto, saya seorang programmer yang baru lulus dari \nUniversitas Hang Tuah di Surabaya di tahun 2024. Saya mengambil jurusan Teknik \ninformatika Bersama 78 teman saya lainnya. Selama kuliah, saya mempelajari banyak hal \nmengenai pemrograman, algorithma, kompleksitas, eﬁsiensi, kolaboratif, dan lain-lainnya \nyang sangat menantang bagi saya. Hal ini karena saya baru mempunyai personal computer \nsaya 5 tahun yang lalu, dan waktu itu computer adalah hal yang benar-benar baru bagi \nsaya. \n \nSaya lahir di Surabaya, dalam keluarga yang kondisi ekonominya kurang baik, bapak saya \nadalah seorang pedagang asongan yang bekerja keras membanting tulang untuk \nmemenuhi kebutuhan hidup saya sebagai anak tunggalnya. Ibu saya juga pekerja paruh \nwaktu di salah satu usaha laundry milik tetangga. Di rumah, kami juga ditemani oleh 

In [41]:
# embeddings =  HuggingFaceEmbeddings(
#         model_name="intfloat/multilingual-e5-large",     # Provide the pre-trained model's path
#         model_kwargs={'device':'cpu'}, # Pass the model configuration options
#         encode_kwargs={'normalize_embeddings': False}, # Pass the encoding options
#         show_progress=True
#     )
embeddings =  HuggingFaceEmbeddings(
        model_name="sentence-transformers/distiluse-base-multilingual-cased-v2",     # Provide the pre-trained model's path
        model_kwargs={'device':'cpu'}, # Pass the model configuration options
        encode_kwargs={'normalize_embeddings': False}, # Pass the encoding options
        show_progress=True
    )

In [10]:
db = FAISS.from_documents(documents=documents, embedding=embeddings, ids=uuids)

Batches: 100%|██████████| 1/1 [00:00<00:00,  3.42it/s]


In [11]:
db.docstore._dict

{'fd1214ca-758f-4dec-9904-3a720a14a2d9': Document(id='fd1214ca-758f-4dec-9904-3a720a14a2d9', metadata={'parent_id': 'c439b32a-9e4e-4805-acef-21a7daa0510e', 'source': '../dataset/RH_Budi_Casual.pdf', 'page': 0}, page_content='Nama saya adalah Budi Susanto, saya seorang programmer yang baru lulus dari \nUniversitas Hang Tuah di Surabaya di tahun 2024. Saya mengambil jurusan Teknik \ninformatika Bersama 78 teman saya lainnya. Selama kuliah, saya mempelajari banyak hal \nmengenai pemrograman, algorithma, kompleksitas, eﬁsiensi, kolaboratif, dan lain-lainnya \nyang sangat menantang bagi saya. Hal ini karena saya baru mempunyai personal computer \nsaya 5 tahun yang lalu, dan waktu itu computer adalah hal yang benar-benar baru bagi \nsaya. \n \nSaya lahir di Surabaya, dalam keluarga yang kondisi ekonominya kurang baik, bapak saya \nadalah seorang pedagang asongan yang bekerja keras membanting tulang untuk \nmemenuhi kebutuhan hidup saya sebagai anak tunggalnya. Ibu saya juga pekerja paruh \nw

In [17]:
question = "carikan materi yang berkaitan dengan budi"
resultE5Indo = db.similarity_search(question, k=10, filter={"parent_id": {"$in": "c439b32a-9e4e-4805-acef-21a7daa0510e"}})
resultE5Indo

Batches: 100%|██████████| 1/1 [00:00<00:00, 31.31it/s]


[Document(id='fd1214ca-758f-4dec-9904-3a720a14a2d9', metadata={'parent_id': 'c439b32a-9e4e-4805-acef-21a7daa0510e', 'source': '../dataset/RH_Budi_Casual.pdf', 'page': 0}, page_content='Nama saya adalah Budi Susanto, saya seorang programmer yang baru lulus dari \nUniversitas Hang Tuah di Surabaya di tahun 2024. Saya mengambil jurusan Teknik \ninformatika Bersama 78 teman saya lainnya. Selama kuliah, saya mempelajari banyak hal \nmengenai pemrograman, algorithma, kompleksitas, eﬁsiensi, kolaboratif, dan lain-lainnya \nyang sangat menantang bagi saya. Hal ini karena saya baru mempunyai personal computer \nsaya 5 tahun yang lalu, dan waktu itu computer adalah hal yang benar-benar baru bagi \nsaya. \n \nSaya lahir di Surabaya, dalam keluarga yang kondisi ekonominya kurang baik, bapak saya \nadalah seorang pedagang asongan yang bekerja keras membanting tulang untuk \nmemenuhi kebutuhan hidup saya sebagai anak tunggalnya. Ibu saya juga pekerja paruh \nwaktu di salah satu usaha laundry milik t

In [15]:
db.similarity_search(query=question, k=10, filter={"parent_id": {"$in": "c439b32a-9e4e-4805-acef-21a7daa0510e"}})

Batches: 100%|██████████| 1/1 [00:00<00:00, 34.91it/s]


[Document(id='50c7113f-ff6d-4c59-8be2-93152e7d970e', metadata={'parent_id': 'c439b32a-9e4e-4805-acef-21a7daa0510e', 'source': '../dataset/RH_Budi_Casual.pdf', 'page': 0}, page_content='waktu di salah satu usaha laundry milik tetangga. Di rumah, kami juga ditemani oleh \nnenek. Saya sangat dekat dengan nenek saya, karena sebagian besar waktu menghabiskan \nwaktu Bersama dengan beliau, sambil menunggu kepulangan ayah dan ibu dari tempat \nkerjanya masing-masing. Saya bersyukur dengan dukungan orang tua, diizinkan untuk \nmenyelesaikan SD Negeri 5, dilanjutkan SMP Negeri 2 Surabaya. Setelah itu menamatkan \nSMAN 6 Surabaya di tahun 2020. \n \nKondisi sekarang, saya sedang merantau ke kota besar Jakarta untuk mencari pekerjaan, \ndan mulai merajut tangga karir saya. Saya ingin bekerja di sebuah Perusahaan teknologi \n(atau Perusahaan perangkat lunak) karena di topik itulah ilmu yang saya tuntut. Saya sudah \nmelamar ke 17 perusahaan dan sangat bersyukur bahwa dapat mengikuti proses-nya \nd

In [40]:
db.save_local("db_budi")

In [35]:
index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches: 100%|██████████| 1/1 [00:03<00:00,  3.41s/it]


In [36]:
empty_db = FAISS(embedding_function=embeddings, index=index, docstore=InMemoryDocstore(), index_to_docstore_id={})

In [37]:
empty_db.add_documents(documents)

Batches: 100%|██████████| 1/1 [00:04<00:00,  4.18s/it]


['0fa73ffa-f2a8-4201-82e3-bfa069b7aff3',
 'c7f0505e-30d3-4312-832e-04120fe6f449',
 'e1ebbec9-7d51-4945-ba81-65e6c8cc78f5']

In [38]:
question = "carikan materi yang berkaitan dengan riwayat"

In [39]:
empty_db.similarity_search(query=question, k=10, filter={"parent_id": {"$in": "c439b32a-9e4e-4805-acef-21a7daa0510e"}})

Batches: 100%|██████████| 1/1 [00:00<00:00,  2.30it/s]


[Document(id='c7f0505e-30d3-4312-832e-04120fe6f449', metadata={'parent_id': 'c439b32a-9e4e-4805-acef-21a7daa0510e', 'source': '../dataset/RH_Budi_Casual.pdf', 'page': 0}, page_content='waktu di salah satu usaha laundry milik tetangga. Di rumah, kami juga ditemani oleh \nnenek. Saya sangat dekat dengan nenek saya, karena sebagian besar waktu menghabiskan \nwaktu Bersama dengan beliau, sambil menunggu kepulangan ayah dan ibu dari tempat \nkerjanya masing-masing. Saya bersyukur dengan dukungan orang tua, diizinkan untuk \nmenyelesaikan SD Negeri 5, dilanjutkan SMP Negeri 2 Surabaya. Setelah itu menamatkan \nSMAN 6 Surabaya di tahun 2020. \n \nKondisi sekarang, saya sedang merantau ke kota besar Jakarta untuk mencari pekerjaan, \ndan mulai merajut tangga karir saya. Saya ingin bekerja di sebuah Perusahaan teknologi \n(atau Perusahaan perangkat lunak) karena di topik itulah ilmu yang saya tuntut. Saya sudah \nmelamar ke 17 perusahaan dan sangat bersyukur bahwa dapat mengikuti proses-nya \nd